In [11]:
!pip install scikit-optimize
!pip install imblearn
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Machine Learning Models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Balancing of Data
from imblearn.combine import SMOTEENN

# Cross Validation
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.model_selection import StratifiedKFold

# Evaluation
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, RocCurveDisplay, classification_report, f1_score


In [13]:
df = pd.read_csv('/content/final_df.csv')
df

,Patient-Uid,DRUG_TYPE_0,DRUG_TYPE_1,DRUG_TYPE_10,DRUG_TYPE_11,DRUG_TYPE_12,DRUG_TYPE_13,DRUG_TYPE_14,DRUG_TYPE_15,DRUG_TYPE_16,...,SYMPTOM_TYPE_7,SYMPTOM_TYPE_8,SYMPTOM_TYPE_9,TEST_TYPE_0,TEST_TYPE_1,TEST_TYPE_2,TEST_TYPE_3,TEST_TYPE_4,TEST_TYPE_5,Target
0,a0db1e73-1c7c-11ec-ae39-16262ee38c7f,29,0,0,1,0,0,0,0,0,...,1,0,0,10,2,0,0,0,0,0
1,a0dc93f2-1c7c-11ec-9cd2-16262ee38c7f,8,27,0,0,0,0,0,0,0,...,0,0,0,1,4,0,0,0,0,0
2,a0dc94c6-1c7c-11ec-a3a0-16262ee38c7f,6,7,0,10,0,0,0,0,0,...,0,0,0,3,2,0,0,0,0,0
3,a0dc950b-1c7c-11ec-b6ec-16262ee38c7f,15,42,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,a0dc9543-1c7c-11ec-bb63-16262ee38c7f,2,45,0,24,0,0,0,0,0,...,5,6,0,9,27,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27028,a0f0d4c5-1c7c-11ec-bfec-16262ee38c7f,41,7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
27029,a0f0d4f4-1c7c-11ec-b144-16262ee38c7f,18,22,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
27030,a0f0d523-1c7c-11ec-89d2-16262ee38c7f,7,48,0,3,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,1
27031,a0f0d553-1c7c-11ec-a70a-16262ee38c7f,7,44,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1


In [14]:
df.shape

(27033, 57)

In [15]:
X = df.drop(['Patient-Uid', 'Target'], axis=1).values
y = df['Target'].values

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.25, random_state=3)

In [17]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [18]:
pipeline = imbpipeline(steps = [['smote', SMOTEENN(random_state=3)],
                                ['classifier', LogisticRegression(random_state=3,
                                                                  max_iter=1000)]])

stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True,
                                       random_state=3)

param_grid = {'classifier__C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
lr_grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

lr_grid_search.fit(X_train, y_train)
lr_cv_score = lr_grid_search.best_score_
lr_test_score = lr_grid_search.score(X_test, y_test)
print(f'Cross-validation score: {lr_cv_score}\nTest score: {lr_test_score}')

Cross-validation score: 0.6763743393506687
Test score: 0.679831223628692


In [19]:
print(confusion_matrix(y_test, lr_grid_search.predict(X_test)))


[[2848 1567]
 [ 330 2014]]


In [20]:
print(f1_score(y_test, lr_grid_search.predict(X_test)))

0.679831223628692


In [21]:
pipeline = imbpipeline(steps = [['smote', SMOTEENN(random_state=3)],
                                ['classifier', KNeighborsClassifier()]])

stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True,
                                       random_state=3)

param_grid = {'classifier__n_neighbors':np.arange(0, 21, 1)}
knn_grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

knn_grid_search.fit(X_train, y_train)
knn_cv_score = knn_grid_search.best_score_
knn_test_score = knn_grid_search.score(X_test, y_test)
print(f'Cross-validation score: {knn_cv_score}\nTest score: {knn_test_score}')

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
5 fits failed out of a total of 105.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/imblearn/pipeline.py", line 297, in fit
    self._final_estimator.fit(Xt, yt, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py", line 213, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/d

Cross-validation score: 0.6543650842008083
Test score: 0.652482269503546


In [22]:
print(confusion_matrix(y_test, knn_grid_search.predict(X_test)))

[[2674 1741]
 [ 366 1978]]


In [23]:
print(f1_score(y_test, knn_grid_search.predict(X_test)))

0.652482269503546


In [24]:
pipeline = imbpipeline(steps = [['smote', SMOTEENN(random_state=3)],
                                ['classifier', DecisionTreeClassifier(random_state=3)]])

stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True,
                                       random_state=3)

param_grid = {'classifier__max_depth':np.arange(5, 21)}
dt_grid_search = GridSearchCV(estimator = pipeline,
                             param_grid=param_grid,
                             scoring='f1',
                             cv=stratified_kfold,
                             n_jobs=-1)

dt_grid_search.fit(X_train, y_train)
dt_cv_score = dt_grid_search.best_score_
dt_test_score = dt_grid_search.score(X_test, y_test)
print(f'Cross-validation score: {dt_cv_score}\nTest score: {dt_test_score}')

Cross-validation score: 0.6728414485794959
Test score: 0.6899155132122955


In [25]:
print(confusion_matrix(y_test, dt_grid_search.predict(X_test)))

[[3115 1300]
 [ 425 1919]]


In [26]:
print(f1_score(y_test, dt_grid_search.predict(X_test)))

0.6899155132122955


In [ ]:
pipeline = imbpipeline(steps = [['smote', SMOTEENN(random_state=3)],
                                ['classifier', RandomForestClassifier(random_state=3)]])

stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True,
                                       random_state=3)

param_grid = {
    'classifier__n_estimators': [10, 100, 500],
    'classifier__min_samples_split': [2, 3, 4],
    'classifier__min_samples_leaf': [2, 3, 4, 5],
    'classifier__max_depth' : np.arange(5, 26)
}
rf_bayes_search = BayesSearchCV(estimator = pipeline,
                             search_spaces=param_grid,
                             scoring='f1',
                             cv=stratified_kfold,
                             n_jobs=-1)

rf_bayes_search.fit(X_train, y_train)
rf_cv_score = rf_bayes_search.best_score_
rf_test_score = rf_bayes_search.score(X_test, y_test)
print(f'Cross-validation score: {rf_cv_score}\nTest score: {rf_test_score}')

In [ ]:
print(confusion_matrix(y_test, rf_bayes_search.predict(X_test)))

In [ ]:
print(f1_score(y_test, rf_bayes_search.predict(X_test)))

In [ ]:
pipeline = imbpipeline(steps = [['smote', SMOTEENN(random_state=3)],
                                ['classifier', XGBClassifier(random_state=3)]])

stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True,
                                       random_state=3)

param_grid = {
    'classifier__learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
    'classifier__max_depth': [3, 4, 5, 6, 8, 10, 12, 15],
    'classifier__min_child_weight': [1, 3, 5, 7],
    'classifier__gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
    'classifier__colsample_bytree': [0.3, 0.4, 0.5, 0.7]
}
xgb_bayes_search = BayesSearchCV(estimator = pipeline,
                             search_spaces=param_grid,
                             scoring='f1',
                             cv=stratified_kfold,
                             n_jobs=-1)

xgb_bayes_search.fit(X_train, y_train)
xgb_cv_score = xgb_bayes_search.best_score_
xgb_test_score = xgb_bayes_search.score(X_test, y_test)
print(f'Cross-validation score: {xgb_cv_score}\nTest score: {xgb_test_score}')

In [ ]:
print(confusion_matrix(y_test, xgb_bayes_search.predict(X_test)))

In [ ]:
print(f1_score(y_test, xgb_bayes_search.predict(X_test)))

In [ ]:
test_df = pd.read_parquet('/content/drive/MyDrive/Structured_Data_Assignment/test.parquet')

In [ ]:
test_df

In [ ]:
# Pivot like final_df
final_test_df = test_df[['Patient-Uid', 'Incident']].pivot_table(index='Patient-Uid', columns=['Incident'], aggfunc='size', fill_value=0).reset_index()
final_test_df.columns.name = ''

In [ ]:
final_test_df

In [ ]:
final_submission = final_test_df[['Patient-Uid']]

In [ ]:
final_submission

In [ ]:
test_df_fts = final_test_df.drop('Patient-Uid', axis=1)

In [ ]:
test_df_fts

In [ ]:
# Scaling test data
test_df_fts = sc.transform(test_df_fts.values)

In [ ]:
label = xgb_bayes_search.predict(test_df_fts)
label

In [ ]:
final_submission['label'] = label

In [ ]:
final_submission

In [ ]:
final_submission.to_csv('final_submission.csv', index=False)